### Импортируем необходимые библиотеки

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Считываем данные по скорости травления

В данном документе имеются экспериментальные результаты по скоростям травления в зависимости от потоков частиц на поверхность пластины

In [19]:
df = pd.read_csv("etch_rate.csv", sep=';')
df.head()

,ions,Cl,Br,etch_rate
0,1.890000e+15,8.290000e+16,4.090000e+17,225.9
1,1.900000e+15,8.300000e+16,4.080000e+17,228.6
2,1.770000e+15,7.610000e+16,4.060000e+17,228.9
3,1.370000e+15,5.680000e+16,4.040000e+17,220.1
4,1.060000e+15,3.990000e+16,4.030000e+17,215.5


Сущестуют множество сложных моделей, которые позволяют определить скорость травления из физических соображений, однако мы используем методы машинного обучения для простого определения данной зависимости

In [26]:
Ions = np.array(list(df["ions"])).reshape(len(df['ions']),1)
### переведите в удобный формат и остальные параметры

Будем считать, что используется только параметры до степени M. Реализуем нашу модель

In [28]:
def l2pinv(X, lam):
    A = X.T @ X
    B = 2.0*np.diag(np.ones((A.shape[0],)))* lam
    return np.linalg.inv(A + B) @ X.T

def Model(M, Ions, Cl, Br, etch_rate):
    X = np.ones(Ions.shape)
    coeffs = [(0,0,0)] # массив для сохранения коэффициентов
    # добавляем степени ионов
    for k in range(1, M+1):
        coeffs.append((k,0,0))
        X = np.concatenate([X, x_pow_k(Ions, k)], axis = 1)

    ### добавьте степени всех остальных входных параметров, а так же перекрёстные

    print(X.shape)

    # Вычисляем столбец коэффицентов регрессии

    w = l2pinv(X, 0.001) @ etch_rate

    # Смотрим предсказания модели

    etch_rate_predict = X @ w

    return etch_rate_predict, w, coeffs

Проанализируем получившиеся коэффициенты и сделаем вывод относительно того, какие параметры влияют больше всего, а также определим важность перекрёстных коэффициентов

### Задание 1. Напишите функцию, которая будет сравнивать предсказанные скорости и экспериментальные, вычислять среднеквадратичные ошибки: абсолютную и относительные

### Задание 2. Расчитайте коэффициенты для M=1,2,3. И посчитайте ошибки для каждой из них

### Задание 3. Определите наиболее самый важный перекрёстный коэффициент для M=2

### Задание 4. Определите наиболее самый важный перекрёстный коэффициент для M=3

Существует 2 основных механизма травления. Физический, основанный на бомбардировании ионами поверхности пластины, а так же химический, основанный на простой реакции химическиактивных частиц с поверхностью. Каждый из них вносит отдельный вклад, но как они друг с другом взаимодействуют?

### Задание 5. Сделайте вывод относительно совместного влияния физического и химического механизма травления